In [1]:
import sys
import os
import asyncio
import logging
import time

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [2]:
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/sashabrownsberger/ospl.xml"

In [3]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [4]:
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)

In [5]:
await asyncio.gather(attcs.start_task, latiss.start_task)

Read historical data in 17.00 sec
Read historical data in 17.41 sec
Read historical data in 17.74 sec
Read historical data in 18.36 sec
Read historical data in 18.61 sec
Read historical data in 19.95 sec
Read historical data in 20.94 sec
Read historical data in 21.50 sec
Read historical data in 22.07 sec
Read historical data in 23.47 sec
Read historical data in 26.77 sec
RemoteEvent(ATHeaderService, 0, heartbeat) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 10 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 36 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 10 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 11 messages


[[None, None, None, None, None, None, None], [None, None, None, None]]

RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 40 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 10 messages


In [5]:

await attcs.enable()
await latiss.enable() 

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 18 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 10 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 40 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 20 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 20 messages
Gathering settings.
No settings for atmcs.
Using                                                                                                                                 from settingVersions event.
No settings for atptg.
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 21 messages
Couldn't get settingVersions event. Using empty settings.
No settings for ataos.
Using current from settingVersions event.
No settings for atpneumatics.
Using                                                                                                                                 from settingVersions event.
No settings 

In [6]:
# !!!! STOP !!!!
#This command will open the dome and.  Do not run until the sky is sufficiently dark! 
await attcs.startup()
#You may want to verify that the dome is open

Gathering settings.
No settings for atmcs.
Using                                                                                                                                 from settingVersions event.
No settings for atptg.
RemoteEvent(ATMCS, 0, target) falling behind; read 69 messages
Couldn't get settingVersions event. Using empty settings.
No settings for ataos.
Using current from settingVersions event.
No settings for atpneumatics.
Using                                                                                                                                 from settingVersions event.
No settings for athexapod.
Using current from settingVersions event.
No settings for atdome.
Using test from settingVersions event.
No settings for atdometrajectory.
Using  from settingVersions event.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 

CancelledError: 

[Telescope] delta Alt = +000.063 | delta Az= +024.631
[Telescope] delta Alt = -000.000 | delta Az= +020.719
[Telescope] delta Alt = -000.000 | delta Az= +016.942
[Telescope] delta Alt = -000.000 | delta Az= +013.384
[Telescope] delta Alt = +000.000 | delta Az= +010.125
[Telescope] delta Alt = +000.000 | delta Az= +007.252
[Telescope] delta Alt = +000.000 | delta Az= +004.831
[Telescope] delta Alt = +000.000 | delta Az= +002.912
[Telescope] delta Alt = +000.000 | delta Az= +002.147
[Telescope] delta Alt = +000.000 | delta Az= +001.004
[Telescope] delta Alt = +000.000 | delta Az= +000.336
[Telescope] delta Alt = +000.000 | delta Az= +000.050
[Telescope] delta Alt = +000.000 | delta Az= -000.000
Axes in position.


In [7]:
await attcs.stop_all()

Got True
Waiting for telescope to settle.
Telescope in position.


 AckTimeoutError(msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=1847128462, ack=<SalRetCode.CMD_ACK: 300>, error=0, result='SAL ACK'))]

In [8]:
#check if tasks are pending
attcs.scheduled_coro

[<Task pending coro=<ATTCS.wait_for_inposition() running at /home/sashabrownsberger/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py:938> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f47bdb7b150>()]> cb=[as_completed.<locals>._on_completion() at /opt/lsst/software/stack/python/miniconda3-4.7.12/envs/lsst-scipipe-984c9f7/lib/python3.7/asyncio/tasks.py:547]>,
 <Task finished coro=<ATTCS.monitor_position() done, defined at /home/sashabrownsberger/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py:1064> result=None>,
 <Task pending coro=<BaseGroup.check_component_state() running at /home/sashabrownsberger/repos/ts_standardscripts/python/lsst/ts/standardscripts/base_group.py:92> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f47bd983950>()]> cb=[as_completed.<locals>._on_completion() at /opt/lsst/software/stack/python/miniconda3-4.7.12/envs/lsst-scipipe-984c9f7/lib/python3.7/asyncio/tasks.py:547]>,
 <Task pend

In [9]:
await attcs.cancel_not_done(attcs.scheduled_coro)

In [ ]:
await attcs.atdome.cmd_openShutter.start() 

In [7]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200033 2020031200034 2020031200035 2020031200036 2020031

In [8]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200043 2020031200044 2020031200045 2020031200046 2020031200047
 2020031200048 2020031200049 2020031200050 2020031200051 2020031200052]


In [9]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
BIAS 0007 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200053 2020031200054 2020031200055 2020031200056 2020031

In [10]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
The bias data id(s) are: [2020031200063 2020031200064 2020031200065 2020031200066 2020031200067
 2020031200068 2020031200069 2020031200070 2020031200071 2020031200072

In [11]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list))

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
BIAS 0006 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200073 2020031200074 2020031200075 2020031200076 2020031200077
 2020031200078 2020031200079 2020031200080 2020031200081 2020031200082]


In [12]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200083 2020031200084 2020031200085 2020031200086 2020031200087
 2020031200088 2020031200089 2020031200090 2020031200091 2020031200092

In [13]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Waiting for header service LFO before continuing
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200093 2020031200094 2020031200095 2020031200096 2020031200097
 2020031200098 2020031200099 2020031200100 2020031200101 2020031200102]


In [14]:
#Let's take some biases 
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

Generating group_id
BIAS 0001 - 0010
Waiting for header service LFO before continuing
BIAS 0002 - 0010
Waiting for header service LFO before continuing
BIAS 0003 - 0010
Waiting for header service LFO before continuing
BIAS 0004 - 0010
Waiting for header service LFO before continuing
BIAS 0005 - 0010
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
BIAS 0006 - 0010
Waiting for header service LFO before continuing
BIAS 0007 - 0010
Waiting for header service LFO before continuing
BIAS 0008 - 0010
Waiting for header service LFO before continuing
BIAS 0009 - 0010
Waiting for header service LFO before continuing
BIAS 0010 - 0010
Waiting for header service LFO before continuing
The bias data id(s) are: [2020031200103 2020031200104 2020031200105 2020031200106 2020031200107
 2020031200108 2020031200109 2020031200110 2020031200111 2020031200112]


In [15]:
#I'm responsible primarily for slewing the telescope.  Patrick will deal with taking the data 
await attcs.slew_object(name = "HD 34816",  rot_pa=0.0, slew_timeout=240.)

Starting new HTTP connection (1): simbad.u-strasbg.fr:80
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HD 34816: 05 19 34.5240 -13 10 36.440
Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 11 messages
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -010.352 | delta Az= -036.087 [Dome] delta Az = -129.078
[Telescope] delta Alt = -009.331 | delta Az= -033.786 [Dome] delta Az = -128.308
[Telescope] delta Alt = -006.130 | delta Az= -029.811 [Dome] delta Az = -126.818
[Telescope] delta Alt = -002.338 | delta Az= -025.840 [Dome] delta Az = -124.238
[Telescope] delta Alt = -000.224 | delta Az= -02

In [16]:
#And we take a single exposure
filter_str = 'empyt_1' #Optional filters are: 
grating_str = 'empty_1' #Optional gratings are: 
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='empty_1')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200113]


In [23]:
#Try to display image... 
import os
import numpy as np

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend("firefly")
os.environ['FIREFLY_URL'] = "http://firefly01.cp.lsst.org:8080/firefly/"

In [24]:
from lsst.ip.isr.isrTask import IsrTask

isrConfig = IsrTask.ConfigClass()
isrConfig.doLinearize = False
isrConfig.doBias = False
isrConfig.doFlat = False
isrConfig.doDark = False
isrConfig.doFringe = False
isrConfig.doDefect = False
isrConfig.doAddDistortionModel = False
isrConfig.doWrite = False

isrTask = IsrTask(config=isrConfig)

/opt/lsst/software/stack/python/miniconda3-4.7.12/envs/lsst-scipipe-984c9f7/lib/python3.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Config field doAddDistortionModel is deprecated: Camera geometry is incorporated when reading the raw files. This option no longer is used, and will be removed after v19.


In [25]:
butler = dafPersist.Butler("/project/shared/auxTel")
dataId = dict(dayObs="2020-03-12") #Your observing date 

RemoteEvent(ATMCS, 0, target) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 27 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 13 messages


In [20]:
dataId["seqNum"] = 113
exp = butler.get('raw', dataId)

Using translation class LSST_LATISS
Checking for header correction file named LATISS-AT_O_20200312_000113.yaml
AT_O_20200312_000113: Forcing detector serial to ITL-3800C-068
AT_O_20200312_000113: Forcing SHUTTIME header to be None
Using translation class LSST_LATISS
Checking for header correction file named LATISS-AT_O_20200312_000113.yaml
AT_O_20200312_000113: Forcing detector serial to ITL-3800C-068
Using translation class LSST_LATISS
AT_O_20200312_000113: Dark time less than exposure time. Setting dark time to the exposure time.
RemoteEvent(ATMCS, 0, target) falling behind; read 14 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages


In [22]:
disp = afwDisplay.Display(1, name='sample', reopenPlot=True)
disp.scale('asinh', 'zscale')
disp.mtv(exp, title=f"{dataId}")

https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
https://summit-lsp.lsst.codes:443 "POST /portal/app/sticky/CmdSrv HTTP/1.1" 200 38
http

In [ ]:
#Merlin got his display working 

In [27]:
current_pix = np.array((2245, 2216))
target_pix = np.array((1725, 1704))

In [28]:
await attcs.offset_xy(*(target_pix - current_pix),  persistent = True)

Applying x/y offset: -520/ -512 
Applying Az/El offset: -355.2275970363423/ 637.4616492807909 
Waiting for telescope to settle.
Done


In [29]:
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='empty_1')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200114]


In [30]:
await attcs.offset_xy(* (-1 * (target_pix - current_pix)),  persistent = True)
await attcs.offset_xy(*((target_pix - current_pix) / 10),  persistent = True)

Applying x/y offset: 520/ 512 
Applying Az/El offset: 338.12041110978043/ -646.6982198761282 
Waiting for telescope to settle.
Done
Applying x/y offset: -52.0/ -51.2 
Applying Az/El offset: -33.2974108254939/ 64.93629518472918 
Waiting for telescope to settle.
Done


In [31]:
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='empty_1')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200115]


In [32]:
await attcs.offset_xy(x = 0, y = -40, persistent = True)

Applying x/y offset: 0/ -40 
Applying Az/El offset: 14.028666038537507/ 37.45926493111129 
Waiting for telescope to settle.
Done


In [33]:
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='empty_1')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
[2020031200116]


In [35]:
import math
#Let's try to get roughly in focus. 
#Focus units are given in mm 
focus_center = -0.2155
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str(focus_positions))

focus_positions = [-0.2955 -0.2755 -0.2555 -0.2355 -0.2155 -0.1955 -0.1755 -0.1555 -0.1355]


In [36]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    non_dispersed_focus_image = await latiss.take_engtest(exptime=2., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(non_dispersed_focus_image ) )
    non_dispersed_focus_images[i] = int(non_dispersed_focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


NameError: name 'focus_image' is not defined

In [37]:
#Whoops.  With Ronchi grating, position is off.  Let's shift and try again... 
await attcs.offset_xy(1283 - 1515, 0.0, persistent = True)

Applying x/y offset: -232/ 0.0 
Applying Az/El offset: -210.82234887208574/ 96.83975018584349 
Waiting for telescope to settle.
Done


In [39]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    non_dispersed_focus_image = await latiss.take_engtest(exptime=2., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(non_dispersed_focus_image ) )
    non_dispersed_focus_images[i] = int(non_dispersed_focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


TimeoutError: 

In [40]:
#That's an issue that happens if I try to send a focus position to the same position it's already at.  

#adjust by hand 
await attcs.focus_offset(focus_pos)

In [41]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    non_dispersed_focus_image = await latiss.take_engtest(exptime=2., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(non_dispersed_focus_image ) )
    non_dispersed_focus_images[i] = int(non_dispersed_focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)

Working on focus position -0.2955
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
Newest image id is = [2020031200118]


NameError: name 'non_dispersed_focus_images' is not defined

In [42]:
#target off detector 
await attcs.slew_object(name = "HD 34816",  rot_pa=0.0, slew_timeout=240.)

Slewing to HD 34816: 05 19 34.5240 -13 10 36.440
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -000.027 | delta Az= -000.117 [Dome] delta Az = -000.242
[Telescope] delta Alt = +000.002 | delta Az= +000.003 [Dome] delta Az = -000.242
Axes in position.
None


In [44]:
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
[2020031200119]


In [48]:
shift = ( target_pix -(np.array([2163, 2472]))) / 10
print (shift)

[-43.8 -76.8]


In [49]:
await attcs.offset_xy(*shift, persistent = True)

Applying x/y offset: -43.8/ -76.8 
Applying Az/El offset: 13.807802323633217/ 87.32711259964725 
Waiting for telescope to settle.
Done


In [50]:
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200120]
RemoteEvent(ATMCS, 0, target) falling behind; read 40 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages


In [51]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
non_dispersed_focus_images = [0 for focus_pos in focus_positions] 
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    non_dispersed_focus_image = await latiss.take_engtest(exptime=2., n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(non_dispersed_focus_image ) )
    non_dispersed_focus_images[i] = int(non_dispersed_focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)

Working on focus position -0.2955
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Newest image id is = [2020031200121]
Working on focus position -0.27549999999999997
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Newest image id is = [2020031200122]
Working on focus position -0.2555
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
Newest image id is = [2020031200123]
Working on focus position -0.2355
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Newest image id is = [2020031200124]
Working on focus position -0.2155
Generating group_id
ENGTEST 0001 - 0001
Waiting for header service LFO before cont

In [52]:
# That worked okay, though we had atmospheric deviations. 
best_focus = -0.19
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus)

In [53]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200131]
RemoteEvent(ATMCS, 0, target) falling behind; read 19 messages


In [54]:
shift = ( target_pix -(np.array([2349, 2234]))) / 10
print (shift)
await attcs.offset_xy(*shift, persistent = True)

[-62.4 -53. ]
Applying x/y offset: -62.4/ -53.0 
Applying Az/El offset: 57.636839308972476/ 58.14425813845839 
Waiting for telescope to settle.
Done


In [55]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=1., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200132]


In [56]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=20., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200133]


In [57]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=20., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
[2020031200134]
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages


In [ ]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=20., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

In [58]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=20., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200135]


In [59]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=20., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200136]


In [60]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=25., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200137]
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 80 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 25 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 16 messages


In [66]:
# That worked okay, though we had atmospheric deviations. 
best_focus = -0.19
empty_to_quad = +0.0265
empty_to_BG = +0.0227
empty_to_RG = +0.0227 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_quad )

In [63]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=25., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200139]


In [64]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_BG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=25., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200140]


In [67]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_RG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=25., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200141]


In [68]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_RG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=25., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
Waiting for header service LFO before continuing
[2020031200142]
RemoteEvent(ATMCS, 0, target) falling behind; read 38 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages


In [10]:
#target off detector 
await attcs.slew_object(name = "HD 185975",  rot_pa=0.0, slew_timeout=240.)

Starting new HTTP connection (1): simbad.u-strasbg.fr:80
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HD 185975: 20 28 18.7402 -87 28 19.938
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 17 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 16 messages
Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 22 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 22 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 23 messages
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
[Telescope] delta Alt = -052.180 | de

In [11]:
#Because I stopped it, I need to rerun the enable again.  

await attcs.enable()
await latiss.enable() 

Gathering settings.
No settings for atmcs.
Using                                                                                                                                 from settingVersions event.
No settings for atptg.
Couldn't get settingVersions event. Using empty settings.
No settings for ataos.
Using current from settingVersions event.
No settings for atpneumatics.
Using                                                                                                                                 from settingVersions event.
No settings for athexapod.
Using current from settingVersions event.
No settings for atdome.
Using test from settingVersions event.
No settings for atdometrajectory.
Using  from settingVersions event.
Settings versions: {'atmcs': '                                                                                                                               ', 'atptg': '', 'ataos': 'current', 'atpneumatics': '                                              

In [12]:
#And we forgot to open the lower shutter...Do it now. 
await attcs.atdome.cmd_openShutter.start() 

RemoteEvent(ATMCS, 0, target) falling behind; read 22 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 66 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 13 messages


In [13]:
#Take a quick exposure w/ the grating to get a reference position. 
#Put to nominal best fit position 
best_focus = -0.19
empty_to_quad = +0.0265
empty_to_BG = +0.0227
empty_to_RG = +0.0227 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_quad )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200144]
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages


In [14]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')

Generating group_id
OBJECT 0001 - 0001


In [16]:
target_pix = np.array((1725, 1704))
shift = ( target_pix -(np.array([2163, 2472]))) / 10
print (shift)

[-43.8 -76.8]


In [18]:
await attcs.offset_xy(*shift, persistent = True)

Applying x/y offset: -43.8/ -76.8 
Applying Az/El offset: 80.68638233855664/ 36.143985739210564 
Waiting for telescope to settle.
Done


In [19]:
#Did I get the position right? 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')

Generating group_id
OBJECT 0001 - 0001
RemoteEvent(ATMCS, 0, target) falling behind; read 81 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 30 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 15 messages


In [20]:
target_pix = np.array((1725, 1704))
shift = ( target_pix -(np.array([1911, 1511]))) / 10
print (shift)

[-18.6  19.3]


In [21]:
await attcs.offset_xy(*shift, persistent = True)

Applying x/y offset: -18.6/ 19.3 
Applying Az/El offset: -17.166498872661514/ 20.58546371726688 
Waiting for telescope to settle.
Done


In [22]:
#Did I get the position right? 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')

Generating group_id
OBJECT 0001 - 0001


In [23]:
print (science_image)

[2020031200148]


In [ ]:
#Did I get the position right? 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')

In [24]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus )
science_image = await latiss.take_object(exptime=30., n=1, filter='empty_1',grating='ronchi90lpmm')

Generating group_id
OBJECT 0001 - 0001


In [25]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_quad )
science_image = await latiss.take_object(exptime=30., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200150]


In [26]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_BG )
science_image = await latiss.take_object(exptime=30., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200151]


In [27]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus + empty_to_RG )
science_image = await latiss.take_object(exptime=30., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200152]
RemoteEvent(ATMCS, 0, target) falling behind; read 28 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 33 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 31 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, hea

In [28]:
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = best_focus )

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages


In [ ]:
await attcs.ataos.cmd_reset

In [29]:
#HD60753
#target off detector 
await attcs.slew_object(name = "HD 34816",  rot_pa=0.0, slew_timeout=240.)

Slewing to HD 34816: 05 19 34.5240 -13 10 36.440
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
[Telescope] delta Alt = +000.002 | delta Az= +000.009 [Dome] delta Az = -000.231
[Telescope] delta Alt = +000.002 | delta Az= -000.003 [Dome] delta Az = -000.231
Axes in position.
None


In [30]:
#HD60753
#target off detector 
await attcs.slew_object(name = "HD 34816",  rot_pa=74.0, slew_timeout=240.)

Slewing to HD 34816: 05 19 34.5240 -13 10 36.440
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
[Telescope] delta Alt = +000.004 | delta Az= -000.008 [Dome] delta Az = -000.231
[Telescope] delta Alt = +000.002 | delta Az= +000.001 [Dome] delta Az = -000.231
Axes in position.
None
RemoteEvent(ATMCS, 0, target) falling behind; read 20 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 23 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 58 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) 

In [31]:
await attcs.slew_object("HD 116351", rot_pa = 0.0, slew_timeout=240.)

Resetting dropped connection: simbad.u-strasbg.fr
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HD 116351: 13 23 54.2476 -46 12 00.503
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 63 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 64 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 17 messages
Sending command
Stop tracking.
Mount tracking state is 9
RemoteTelemetry(ATDome, 0, position) falling behind; read 16 messages
RemoteEvent(ATDome, 0, doorEncoderExtremes) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 18 messages
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State

In [32]:
await attcs.slew_object("HD 116351", rot_pa = 0.0, slew_timeout=240.)

Slewing to HD 116351: 13 23 54.2476 -46 12 00.503
Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
[Telescope] delta Alt = +000.839 | delta Az= +002.057 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.842 | delta Az= +002.064 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.846 | delta Az= +002.075 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.851 | delta Az= +002.087 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.854 | delta Az= +002.097 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.859 | delta Az= +002.108 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.863 | delta Az= +002.120 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.867 | delta Az= +002.

CancelledError: 

[Telescope] delta Alt = +000.933 | delta Az= +002.307 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.938 | delta Az= +002.319 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.942 | delta Az= +002.331 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.946 | delta Az= +002.342 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.950 | delta Az= +002.353 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.954 | delta Az= +002.364 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.959 | delta Az= +002.375 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.963 | delta Az= +002.386 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.967 | delta Az= +002.398 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.971 | delta Az= +002.409 [Dome] delta Az = +003.012
[Telescope] delta Alt = +000.975 | delta Az= +002.420 [Dome] delta Az = +003.012


In [33]:
await attcs.cancel_not_done(attcs.scheduled_coro)

In [34]:
await attcs.point_azel(140, 68)

Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380


CancelledError: 

[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380
[Telescope] delta Alt = -000.157 | delta Az= -002.380


In [41]:
await attcs.cancel_not_done(attcs.scheduled_coro)

In [63]:
await attcs.point_azel(142.38, 68.16)

Sending command
Stop tracking.
Mount tracking state is 10
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = +000.003 | delta Az= -000.000
Axes in position.
None


In [62]:
await attcs.cancel_not_done(attcs.scheduled_coro)

In [65]:
await attcs.atmcs.cmd_startTracking.start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1773338855, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='ERROR: Command startTracking rejected while in TrackingEnabledState state.')

In [50]:
await attcs.atmcs.cmd_enable.start()

In [66]:
await attcs.slew_object("HD 116351", rot_pa = 0.0, slew_timeout=240.)

Slewing to HD 116351: 13 23 54.2476 -46 12 00.503
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = +002.395 | delta Az= +007.113
[Telescope] delta Alt = +000.649 | delta Az= +004.927
[Telescope] delta Alt = -000.001 | delta Az= +001.244
[Telescope] delta Alt = -000.001 | delta Az= -000.002
Axes in position.
None
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 59 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 20 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 66 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling 

In [67]:
#Now take some science data.  Same set of loops as before. 
await attcs.slew_object("HD 185975", rot_pa = 0.0, slew_timeout=240.)

Slewing to HD 185975: 20 28 18.7402 -87 28 19.938
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
Got False
Telescope not in position
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
[Telescope] delta Alt = -031.683 | delta Az= -016.619
[Telescope] delta Alt = -028.679 | delta Az= -016.130
[Telescope] delta Alt = -024.698 | delta Az= -014.237
[Telescope] delta Alt = -020.809 | delta Az= -010.968
[Telescope] delta Alt = -017.090 | delta Az= -007.040
[Telescope] delta Alt = -013.620 | delta Az= -003.434
[Telescope] delta Alt = -010.472 | delta Az= -001.004
[Telescope] delta Alt = -007.696 | delta Az= -000.063
[Telescope] delta Alt = -005.345 | delta Az= -000.000
[Telescope] delta Alt = -003.440 | delta Az= +000.000
[Telescope] delta Alt = -001.995 | delta Az= -

In [69]:
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200295]


In [70]:
target_pix = np.array((1770, 1704))
shift = ( target_pix -(np.array([1839, 1930]))) / 10
print (shift)

[ -6.9 -22.6]


In [71]:
await attcs.offset_xy(*shift, persistent = True)

Applying x/y offset: -6.9/ -22.6 
Applying Az/El offset: 1.765222627860761/ 23.56382798006488 
Waiting for telescope to settle.
Done


In [72]:
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200296]


In [75]:
#Focus units are given in mm 
focus_center = -0.15
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * np.floor(n_steps / 2), focus_center + focus_step * np.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str([pos for pos in focus_positions]))

focus_positions = [-0.22999999999999998, -0.21, -0.19, -0.16999999999999998, -0.15, -0.13, -0.11, -0.09, -0.06999999999999999]


In [77]:
await attcs.focus_offset(focus_center)

In [78]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    focus_image = await latiss.take_engtest(exptime=10, n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(focus_image ) )
    focus_images[i] = int(focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)

Working on focus position -0.22999999999999998
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200297]
Working on focus position -0.21
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200298]
Working on focus position -0.19
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200299]
Working on focus position -0.16999999999999998
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200300]
Working on focus position -0.15
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200301]
Working on focus position -0.13
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200302]
Working on focus position -0.11
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200303]
Working on focus position -0.09
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Generating group_id
ENGTEST 0001 - 0001
RemoteEvent(ATHeaderService, 0, logMessage) falling behind

In [79]:
best_fit = -0.12

In [80]:
#I missed. Let's try to do better... 
focus_center = best_fit
focus_step = 0.005
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * np.floor(n_steps / 2), focus_center + focus_step * np.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str([pos for pos in focus_positions]))

focus_positions = [-0.13999999999999999, -0.13499999999999998, -0.12999999999999998, -0.12499999999999999, -0.12, -0.11499999999999999, -0.10999999999999999, -0.10499999999999998, -0.09999999999999999]


In [81]:
await attcs.focus_offset(focus_center)

In [82]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    focus_image = await latiss.take_engtest(exptime=10, n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(focus_image ) )
    focus_images[i] = int(focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)

Working on focus position -0.13999999999999999
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200306]
Working on focus position -0.13499999999999998
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200307]
Working on focus position -0.12999999999999998
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200308]
Working on focus position -0.12499999999999999
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200309]
Working on focus position -0.12
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200310]
Working on focus position -0.11499999999999999
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200311]
Working on focus position -0.10999999999999999
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200312]
Working on focus position -0.10499999999999998
Generating group_id
ENG

In [83]:
best_fit = -0.115
await attcs.focus_offset(best_fit)

In [84]:
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200315]


In [85]:
# That worked okay, though we had atmospheric deviations. 
best_focus = -0.19
empty_to_quad = +0.0265
empty_to_BG = +0.0227
empty_to_RG = +0.0227 

In [86]:
await attcs.focus_offset(best_focus + empty_to_quad )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200316]


In [87]:
await attcs.focus_offset(best_focus + empty_to_BG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200317]


In [88]:
await attcs.focus_offset(best_focus + empty_to_RG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200318]


In [89]:
focus_center = best_fit
focus_step = 0.005
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * np.floor(n_steps / 2), focus_center + focus_step * np.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str([pos for pos in focus_positions]))


focus_positions = [-0.135, -0.13, -0.125, -0.12000000000000001, -0.115, -0.11, -0.10500000000000001, -0.1, -0.095]


In [90]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    focus_image = await latiss.take_engtest(exptime=20, n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(focus_image ) )
    focus_images[i] = int(focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)

Working on focus position -0.135
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200319]
Working on focus position -0.13
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200320]
Working on focus position -0.125
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200321]
Working on focus position -0.12000000000000001
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200322]
Working on focus position -0.115
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200323]
Working on focus position -0.11
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200324]
Working on focus position -0.10500000000000001
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200325]
Working on focus position -0.1
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Generating group_id

In [92]:
#A little better 
best_focus = -0.11

In [93]:
await attcs.focus_offset(best_focus  )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=120., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200328]


In [94]:
await attcs.focus_offset(best_focus + empty_to_quad )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=120., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200329]


In [97]:
await attcs.focus_offset(best_focus + empty_to_BG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=120., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

RemoteEvent(ATMCS, 0, target) falling behind; read 12 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 27 messages
Generating group_id
OBJECT 0001 - 0001
RemoteEvent(ATMCS, 0, target) falling behind; read 41 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 13 messages
[2020031200330]


In [99]:
await attcs.focus_offset(best_focus + empty_to_RG )
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=120., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200331]
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [ ]:
#Now we slew to a new, rising target: 
await attcs.slew_object("HD 146233", rot_pa = 0.0, slew_timeout=240.)

In [ ]:
#Let's center on target 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

In [100]:
focus_center = best_fit
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * np.floor(n_steps / 2), focus_center + focus_step * np.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str([pos for pos in focus_positions]))

focus_positions = [-0.19, -0.17, -0.15, -0.13, -0.11, -0.09, -0.07, -0.04999999999999999, -0.03]


In [101]:
#Let's center on target 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200340]


In [102]:
target_pix = np.array((1770, 1704))
shift = ( target_pix -(np.array([1603, 1751]))) / 10
print (shift)

[16.7 -4.7]


In [103]:
await attcs.offset_xy(*shift, persistent = True)

Applying x/y offset: 16.7/ -4.7 
Applying Az/El offset: 5.052457738367569/ -16.596766877979867 
Waiting for telescope to settle.
Done


In [104]:
#Let's center on target 
science_image = await latiss.take_object(exptime=2., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200341]


In [105]:
focus_center = -0.20
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * np.floor(n_steps / 2), focus_center + focus_step * np.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str([pos for pos in focus_positions]))

focus_positions = [-0.28, -0.26, -0.24000000000000002, -0.22000000000000003, -0.2, -0.18, -0.16, -0.14, -0.12000000000000001]


In [106]:
#!!!!DO NOT RUN UNLESS YOU HAVE BEEN CLEARED TO CONTROL TELESCOPE!!!!! 
focus_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    await attcs.focus_offset(focus_pos)
    focus_image = await latiss.take_engtest(exptime=10, n=1, filter='empty_1', grating='ronchi90lpmm') 
    print ('Newest image id is = ' + str(focus_image ) )
    focus_images[i] = int(focus_image )
    
#This resets the focus position after taking this sequence: 
await attcs.focus_offset(focus_pos)

Working on focus position -0.28
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200342]
Working on focus position -0.26
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200343]
Working on focus position -0.24000000000000002
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200344]
Working on focus position -0.22000000000000003
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200345]
Working on focus position -0.2
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200346]
Working on focus position -0.18
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200347]
Working on focus position -0.16
Generating group_id
ENGTEST 0001 - 0001
Newest image id is = [2020031200348]
Working on focus position -0.14
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Generating group_id
EN

In [107]:
best_focus = -0.20
await attcs.focus_offset(best_focus)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200351]


In [109]:
await attcs.focus_offset(best_focus + empty_to_quad)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

RemoteEvent(ATMCS, 0, target) falling behind; read 61 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
Generating group_id
OBJECT 0001 - 0001
[2020031200352]


In [110]:
await attcs.focus_offset(best_focus + empty_to_BG)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200353]


In [111]:
await attcs.focus_offset(best_focus + empty_to_RG)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200354]


In [112]:
best_focus = -0.20
await attcs.focus_offset(best_focus)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='empty_1',grating='ronchi90lpmm')
print (science_image)


Generating group_id
OBJECT 0001 - 0001
[2020031200355]


In [113]:
await attcs.focus_offset(best_focus + empty_to_quad)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='quadnotch1',grating='ronchi90lpmm')
print (science_image)

Generating group_id
OBJECT 0001 - 0001
[2020031200356]


In [114]:
await attcs.focus_offset(best_focus + empty_to_BG)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='BG40',grating='ronchi90lpmm')
print (science_image)

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
Generating group_id
OBJECT 0001 - 0001
[2020031200357]


In [115]:
await attcs.focus_offset(best_focus + empty_to_RG)
#They also moved the star from underme, so I need to get it recentered
science_image = await latiss.take_object(exptime=60., n=1, filter='RG610',grating='ronchi90lpmm')
print (science_image)

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Generating group_id
OBJECT 0001 - 0001
[2020031200358]
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


# Now we slew to a new, rising target: 
await attcs.slew_object("HD 185975", rot_pa = 0.0, slew_timeout=240.)